In [ ]:
import torch
import os
from gomoku import Gomoku, PLAYER1
from model import ResNet
from training import AlphaZero
from mcts import AlphaMCTS

torch.manual_seed(0)

In [ ]:
def in_colab():
    try:
        import google.colab
        return True
    except ImportError:
        return False

if in_colab():
    from google.colab import drive
    drive.mount('/content/drive')
    base_path = '/content/drive/My Drive/Colab Notebooks'
else:
    base_path = '.'

model_path = os.path.join(base_path, 'alpha_gomoku/model')
optimizer_path = os.path.join(base_path, 'alpha_gomoku/optimizer')

os.makedirs(model_path, exist_ok=True)
os.makedirs(optimizer_path, exist_ok=True)

In [ ]:
BOARD_SIZE = 10

game = Gomoku(BOARD_SIZE)
num_blocks = 16
num_hidden = 256
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = ResNet(game.size, num_blocks, num_hidden, device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-4)
args = {
    'C': 2,
    'num_searches': 500,
    'num_iterations': 3,
    'num_self_play_iterations': 200,
    'num_epochs': 4,
    'num_parallel_games': 200,
    'batch_size': 128,

    'model_path': model_path,
    'optimizer_path': optimizer_path
}

alphaZero = AlphaZero(model, optimizer, game, args)
alphaZero.learn()

In [ ]:
game = Gomoku(BOARD_SIZE)

state = game.get_initial_state()
actions = [0, 7, 1, 8, 2, 9, 3, 10]
player = PLAYER1
for action in actions:
    state = game.get_next_state(state, action, player)
    player = game.get_opponent(player)

game.print(state)

tensor_state = torch.tensor(game.get_encoded_state(state)).unsqueeze(0)

model = ResNet(game.size, num_blocks, num_hidden, device)
model.load_state_dict(torch.load(f'{model_path}/model_{args["num_iterations"] - 1}.pt'))
model.eval()

policy, value = model(tensor_state)
value = value.item()
policy = torch.softmax(policy, axis=1).squeeze(0).detach().cpu().numpy()

print(value, policy)